<a href="https://colab.research.google.com/github/DavidSchineis/Math-Physics/blob/main/Copy_of_Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstract
This lab explores using calculus tools in Python to analyze vector fields. We began by using np.gradient and np.trapezoid to approximate derivatives and integrals and compared them to pen-and-paper solutions. We then extended these methods to 3D fields, calculating gradients, divergence, and curl. We visualized gradients and divergence with quiver and colormesh plots, and then switched to Plotly to display vector fields, curls, and divergence in 3D. Throughout, we verified when values matched our by-hand calculations. This lab demonstrates how calculus and visualization tools can be combined to explore the structure and properties of vector fields in multiple dimensions.

In [ ]:
import numpy as np
from numpy import pi, sin, cos
import time
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Derivative

It is possible to do calculus operations numerically. For example, we can define an arbitrary function y, evaluate it over a range of x values, and to take a derivative of it using np.gradient function. Confirm that it is accurate by integrating the function yourself.

In [ ]:
dx=0.01
x=np.arange(0,10,dx)
y=2*x**2+1

dy=np.gradient(y,x)

plt.plot(x,y,label='function',lw=6)
plt.plot(x,dy,label='np.gradient derivative',lw=10)
plt.plot(x,4*x,lw=3,label='pen-and-paper derivative function')

plt.legend()

plt.show()



###Caption
This plots y=2x^2 + 1 alongside its derivative calculated by np.gradient (orange) and by hand (green). There is clear overlap, so we have confirmed np.gradient's accuracy.

## Integral

Similarly, we can use np.trapezoid function to numerically add up the area under the curve to get an integral. The precision of it depends strongly on the step size we use, the smaller the better (but also slower to compute). Faster and more precise routines such as scipy quad function also exist, but require a somewhat more complex set up.

In [ ]:
I = (2/3)*(10**3) + 10

print('The value of the integral computed using pen-and-paper integration is', I)

for dx in [0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001]:
    t=time.time()
    x=np.arange(0,10,dx)
    a,b=2,1
    y=a*x**2+b
    numI=np.trapezoid(y,x)

    print('The value of the integral using numpy of y from 0 to 10 with dx='+str(dx)+' is',numI)
    print('The precision is',np.abs(numI-I)/I,"%")
    print('It took', time.time()-t,'seconds to compute')
    print('')

To return a function of integral as a function of x, a somewhat more complex array manipulation is needed.

Use a for loop to integrate from 0th to ith element of y (written as y[0:i], with the corresponding x[0:i] to integrate with respect to). Store each calculation in the ith element of the empty Ix array.

In [ ]:
dx=0.01
x=np.arange(0,10,dx)
y=2*x**2+1

Ix=np.zeros(len(x))
for i in range(1, len(x)):
  Ix[i] = np.trapezoid(y[0:i], x[0:i])
plt.plot(x,y,label='function',lw=6)
plt.plot(x,Ix,label='scipy quad integral',lw=10)
plt.plot(x, ((2/3)*(x**3) + x),lw=3,label='pen-and-paper integral function')
plt.legend()
plt.show()

###Caption
This plots y=2x^2 + 1 alongside its integral function calculated by np.trapezoid (orange) and by hand (green).

## Plot the function cos x from 0 to $2\pi$, as well as the corresponding integral and derivative

In [ ]:
x = np.linspace(0,2*pi)
y = cos(x)

dy = -sin(x)
Iy = sin(x)

plt.plot(x,y,label='function',lw=6)
plt.plot(x,dy,label='pen-and-paper derivative',lw=10)
plt.plot(x,Iy,label='pen-and-paper integral',lw=10)

plt.legend()

plt.show()

###Caption
This plots the 2D function of cos from 0 to 2pi as well as its derivative (-sin) and its integral (sin) functions.

## Gradient

It is also possible to do numeric partial integration and differentiation in a similar manner.

Let $f(x,y)=x-y$

We'll first create linearly separated set of points for x & y. We'll then create a meshgrid to diplay the function on a 2d plane.

By default meshgrid is using 'xy' indexing, which results in inverted ordering of the dimensions, which would make x to be along axis=1, and y along axis=0. 'ij' allows to correlate the first axis with x, second axis with y, etc,
which would make it easier to keep track of.

We will then define our function fxy, that would depend on the 2d x & y.

In [ ]:
n=10
x,y=np.linspace(-5,5,n),np.linspace(-5,5,n)
xx,yy = np.meshgrid(x,y, indexing='ij')


fxy=xx-yy

u = np.gradient(fxy,x,axis=0) #partial derivative with respect to x
v = np.gradient(fxy,y,axis=1)

c=plt.pcolormesh(xx, yy, fxy,cmap='rainbow')
plt.colorbar(c,label='f(x,y)')
plt.quiver(xx,yy,u,v,label='Gradient')
plt.legend()
plt.show()

###Caption
This plots f(x,y) = x-y scalar field as a colormap with its gradient vectors plotted as arrows (black). Note that all arrows point down (-y) and to the right (+x) as expected.


## Assuming that $\vec{v}(x,y)=-y\hat{x}+x\hat{y}$, calculate the divergence of this function

In [ ]:
n=6
x,y=np.linspace(-1,1,n),np.linspace(-1,1,n)

xx,yy = np.meshgrid(x,y, indexing='ij')

vx= -yy
vy= xx

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1))

#displaying what this looks like
c=plt.pcolormesh(xx, yy, div,cmap='rainbow',vmin=-1,vmax=3)
plt.colorbar(c,label='divergence')


plt.quiver(xx,yy,vx,vy,label='v(x,y)')
plt.legend()
plt.show()

###Caption
This plots v(x,y) = -y(xhat) + x(yhat) vector field using arrows (black) with its divergence displayed as a colormap. Note that the divergence is 0 everywhere, as expected.

## Curl

Calculating curl will add z dimension, so we might as well handle it explicitly, setting $\vec{v_z}=0$ (but keeping the same shape as $\vec{v_x}$ and $\vec{v_y}$, this is important!), otherwise we will keep $\vec{v}$ the same as earlier.

The code below would make a plot of our vector $\vec{v}$ at every point we computed.

### Numerically calculate the curl, and add a second trace with the vector corresponding to it. Remember that

$$\vec{\nabla}\times\vec{v}=\left(\frac{\partial v_z}{dy}-\frac{\partial v_y}{dz}\right)\hat{x}+\left(\frac{\partial v_x}{dz}-\frac{\partial v_z}{dx}\right)\hat{y}+\left(\frac{\partial v_y}{dx}-\frac{\partial v_x}{dy}\right)\hat{z}$$

Set its colorscale to 'bluered_r'

As a result, you should have two set of vectors - in red you'll display the original function, and in blue you'll display the curl.

In [ ]:
n=6
x,y,z=np.linspace(-1,1,n),np.linspace(-1,1,n),np.linspace(0,1,2)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx= -yy
vy= xx
vz= vx*0 # same shape as vx, but setting every element to zero.



#flatten allows transforming from 2d or 3d arrays to just 1d, which is what is required by the Cone function
fig = go.Figure()
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=vx.flatten(),
    v=vy.flatten(),
    w=vz.flatten(),
    colorscale='bluered',
    cmin=0,
    cmax=0.0001,
    name='v(x,y,z)',
    sizeref=0.1,
    showlegend=True,
    showscale=False))

cx=(np.gradient(vz,y,axis=1) - np.gradient(vy,z,axis=2))
cy=(np.gradient(vx,z,axis=2) - np.gradient(vz,x,axis=0))
cz=(np.gradient(vy,x,axis=0) - np.gradient(vx,y,axis=1))

'''
print(cx)
print(cy)
print(cz)
'''

#Add a trace showing curl vectors
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=cx.flatten(),
    v=cy.flatten(),
    w=cz.flatten(),
    colorscale='bluered_r',
    cmin=0,
    cmax=0.0001,
    name='Curl of v(x,y,z)',
    sizeref=0.1,
    showlegend=True,
    showscale=False))

fig.show()

###Caption
This plots v(x,y,z) = -y(xhat) + x(yhat) vector field using red cones with its curl vector field using blue cones. The curl vector field is constant (2) pointing in positive (zhat).

## Repeat finding divergence and curl of a different vector $\vec{v}(x,y)=x\hat{x}+y\hat{y}$

In [ ]:
n=6
x,y=np.linspace(-1,1,n),np.linspace(-1,1,n)

xx,yy = np.meshgrid(x,y, indexing='ij')

vx= xx
vy= yy

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1))

#displaying what this looks like
c=plt.pcolormesh(xx, yy, div,cmap='rainbow',vmin=-1,vmax=3)
plt.colorbar(c,label='divergence')


plt.quiver(xx,yy,vx,vy,label='v(x,y)')
plt.legend()
plt.show()

### Caption
This plots v(x,y) = x(xhat) + y(yhat) vector field using arrows (black) with its divergence displayed as a colormap. Note that the divergence is 2 everywhere, as expected. Also, that the v(x,y) field expands outwards.

In [ ]:
n=6
x,y,z=np.linspace(-1,1,n),np.linspace(-1,1,n),np.linspace(0,1,2)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx= xx
vy= yy
vz= vx*0 # same shape as vx, but setting every element to zero.



#flatten allows transforming from 2d or 3d arrays to just 1d, which is what is required by the Cone function
fig = go.Figure()
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=vx.flatten(),
    v=vy.flatten(),
    w=vz.flatten(),
    colorscale='bluered',
    cmin=0,
    cmax=0.0001,
    name='v(x,y,z)',
    sizeref=0.1,
    showlegend=True,
    showscale=False))

cx=(np.gradient(vz,y,axis=1) - np.gradient(vy,z,axis=2))
cy=(np.gradient(vx,z,axis=2) - np.gradient(vz,x,axis=0))
cz=(np.gradient(vy,x,axis=0) - np.gradient(vx,y,axis=1))


#Add a trace showing curl vectors
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=cx.flatten(),
    v=cy.flatten(),
    w=cz.flatten(),
    colorscale='bluered',
    cmin=0,
    cmax=0.0001,
    name='Curl of v(x,y,z)',
    sizeref=0.2,
    showlegend=True,
    showscale=False))

fig.show()

### Caption
This plots v(x,y,z) = x(xhat) + y(yhat) vector field using red cones with its curl vector field using blue cones. Note that we've scaled up the sizeref for the curl vectors, but you will stil see that their norms are all near zero, as expected.

#### Question

Calculate the divergence and the curl of both of these vectors algebreically. Do the numerical derivatives agree with your calculations? Pay particular attention to the values that are supposed to be 0.

#### Answer
Divergence of v1 = 0. Curl of v1 = (0,0,2). Divergence of v2 = 2. Curl of v2 = (0,0,0). These divergences and curls match what we expect from our preivous computations/visualizations. This explains why the divergence color maps were a constant color (0 for v1, 2 for v2). It also explains why the curl vector field was entirely 2(zhat) for v1 and why it was entirely near zero for v2.

----
In these cases, curl was pointed in $\hat{z}$ direction only because $\vec{v}$ was positioned solely in $\hat{x}$ and $\hat{y}$ directions. This is of course not always the case.

### Plot the curl of a vector $$\vec{v}=xz\hat{x}+xy\hat{y}+yz\hat{z}$$

Redefine z so that it also goes from -1 to 1 n times.
You may need to experiment with sizeref term in plotting

In [ ]:
n=6
x,y,z=np.linspace(-1,1,n),np.linspace(-1,1,n),np.linspace(-1,1,n)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx= xx*zz
vy= xx*yy
vz= yy*zz



#flatten allows transforming from 2d or 3d arrays to just 1d, which is what is required by the Cone function
fig = go.Figure()
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=vx.flatten(),
    v=vy.flatten(),
    w=vz.flatten(),
    colorscale='bluered',
    cmin=0,
    cmax=0.0001,
    name='v(x,y,z)',
    sizeref=0.3,
    showlegend=True,
    showscale=False))

cx=(np.gradient(vz,y,axis=1) - np.gradient(vy,z,axis=2))
cy=(np.gradient(vx,z,axis=2) - np.gradient(vz,x,axis=0))
cz=(np.gradient(vy,x,axis=0) - np.gradient(vx,y,axis=1))


#Add a trace showing curl vectors
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=cx.flatten(),
    v=cy.flatten(),
    w=cz.flatten(),
    colorscale='bluered_r',
    cmin=0,
    cmax=0.0001,
    name='Curl of v(x,y,z)',
    sizeref=0.2,
    showlegend=True,
    showscale=False))

fig.show()

### Caption
This plots v(x,y,z) = xz(xhat) + xy(yhat) + yz(zhat) vector field using red cones with its curl vector field using blue cones.

#### Question
Experiment with curls of different functions. Is curl always perpendicular to the vector?

In [ ]:
n=6
x,y,z=np.linspace(-1,1,n),np.linspace(-1,1,n),np.linspace(-1,1,n)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx= (xx**2)
vy= (xx*yy*zz)
vz= 3*(zz**3) + 2



#flatten allows transforming from 2d or 3d arrays to just 1d, which is what is required by the Cone function
fig = go.Figure()
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=vx.flatten(),
    v=vy.flatten(),
    w=vz.flatten(),
    colorscale='bluered',
    cmin=0,
    cmax=0.0001,
    name='v(x,y,z)',
    sizeref=0.1,
    showlegend=True,
    showscale=False))

cx=(np.gradient(vz,y,axis=1) - np.gradient(vy,z,axis=2))
cy=(np.gradient(vx,z,axis=2) - np.gradient(vz,x,axis=0))
cz=(np.gradient(vy,x,axis=0) - np.gradient(vx,y,axis=1))


#Add a trace showing curl vectors
fig.add_trace(go.Cone(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    u=cx.flatten(),
    v=cy.flatten(),
    w=cz.flatten(),
    colorscale='bluered_r',
    cmin=0,
    cmax=0.0001,
    name='Curl of v(x,y,z)',
    sizeref=0.3,
    showlegend=True,
    showscale=False))

#fig.show()

####Answer

No, the curl is not always perpendicular to the vector field. In some cases, particularly where the vector field is within a plane (our previous 2D vectors), the curl vector field will be perpendicular, but not all cases.

----
## Extra Credit

Plot divergence of vector $\vec{v}=xz\hat{x}+xy\hat{y}+yz\hat{z}$ in 3d using iso-surfaces. Refer to this tuturial to how it can be set up

https://plotly.com/python/3d-isosurface-plots/

Set surface_count to 20.

In [ ]:
import plotly.graph_objects as go

n=6
x,y,z=np.linspace(-1,1,n),np.linspace(-1,1,n),np.linspace(-1,1,n)
xx,yy,zz = np.meshgrid(x,y,z, indexing='ij')

vx= xx*zz
vy= xx*yy
vz= yy*zz

div= (np.gradient(vx,x,axis=0) + np.gradient(vy,y,axis=1) + np.gradient(vz,z,axis=2))

fig= go.Figure(data=go.Isosurface(
    x=xx.flatten(),
    y=yy.flatten(),
    z=zz.flatten(),
    value=div.flatten(),
    surface_count=20,
    showlegend=True
))

fig.show()

### Caption
This is a 3D isosurface plot of the divergence of the vector field v = xz(xhat) + xy(yhat) + yz(zhat) which evaluates to x+y+z and as seen in the visualization, it is essentially a colormapped summing of the unit cube for the cartesian system.